In [1]:
%load_ext autoreload
%autoreload 2

In [51]:
!pwd

/home/sid/ds_nudge_up


In [63]:
import os
os.environ['USE_PYGEOS'] = "0"

import matplotlib.pyplot as plt

In [55]:
import src.utils as utl
from src.featurize import *

In [28]:
rasterio_config = utl.load_yaml_config("rasterioc_config.yaml")
os.environ.update(rasterio_config)

In [46]:
featurization_params = utl.load_yaml_config("featurisation.yaml")
satellite_config = utl.load_yaml_config("satellite_config.yaml")
satellite_config = satellite_config[featurization_params['satellite_image_params']['satellite_name']]
data_sources = utl.load_yaml_config('data_catalog.yaml')

In [56]:
satellite_config

{'resolution': 10,
 'bands': ['B02,',
  'B03,',
  'B04,',
  'B05,',
  'B06,',
  'B07,',
  'B08,',
  'B11,',
  'B12,'],
 'buffer_distance': 1200,
 'min_image_edge': 90}

In [21]:
featurization_params

{'satellite_image_params': {'satellite_name': 'landsat-8-c2-l2',
  'search_start': datetime.date(2021, 4, 1),
  'search_end': datetime.date(2022, 3, 31),
  'stac_output': 'least_cloudy'},
 'num_features': 4000,
 'dask': {'n_partitions': 250}}

In [24]:
points_gdf = utl.load_points_gdf(**data_sources['request_points'])

In [47]:
%%time
points_gdf_with_stac = fetch_image_refs(points_gdf, featurization_params['dask']['n_partitions'], 
                                        featurization_params['satellite_image_params'])

CPU times: user 7min 4s, sys: 13.9 s, total: 7min 18s
Wall time: 6min 37s


In [89]:
points_gdf_with_stac['stac_item']

91057    <Item id=S2B_MSIL2A_20220129T053049_R105_T43PD...
91056    <Item id=S2B_MSIL2A_20220129T053049_R105_T43PD...
91062    <Item id=S2B_MSIL2A_20220129T053049_R105_T43PD...
89896    <Item id=S2B_MSIL2A_20220129T053049_R105_T43PD...
89898    <Item id=S2B_MSIL2A_20220129T053049_R105_T43PD...
                               ...                        
99726    <Item id=S2B_MSIL2A_20220221T035759_R004_T46NE...
99722    <Item id=S2B_MSIL2A_20220221T035759_R004_T46NE...
99720    <Item id=S2B_MSIL2A_20220122T040049_R004_T46NE...
99719    <Item id=S2B_MSIL2A_20220122T040049_R004_T46NE...
99718    <Item id=S2B_MSIL2A_20220122T040049_R004_T46NE...
Name: stac_item, Length: 99897, dtype: object

In [70]:
stac_item_list = points_gdf_with_stac.stac_item.tolist()
points_list = points_gdf_with_stac[["Lon", "Lat"]].to_numpy()

In [85]:
stac_item_list[0]

ID: S2B_MSIL2A_20220129T053049_R105_T43PDR_20220213T192200
Datetime: 2022-01-29 05:30:49.024000+00:00
datetime: 2022-01-29T05:30:49.024000Z
platform: Sentinel-2B
proj:epsg: 32643
instruments: ['msi']
s2:mgrs_tile: 43PDR
constellation: Sentinel 2
s2:granule_id: S2B_OPER_MSI_L2A_TL_ESRI_20220213T192202_A025585_T43PDR_N04.00
eo:cloud_cover: 0.001353
s2:datatake_id: GS2B_20220129T053049_025585_N04.00


In [81]:
import stackstac

In [83]:
stackstac.stack(stac_item_list[0], satellite_config['bands'],satellite_config['resolution'])

TypeError: cannot unpack non-iterable NoneType object

In [80]:
stac_item_list[0]

ID: S2B_MSIL2A_20220129T053049_R105_T43PDR_20220213T192200
Datetime: 2022-01-29 05:30:49.024000+00:00
datetime: 2022-01-29T05:30:49.024000Z
platform: Sentinel-2B
proj:epsg: 32643
instruments: ['msi']
s2:mgrs_tile: 43PDR
constellation: Sentinel 2
s2:granule_id: S2B_OPER_MSI_L2A_TL_ESRI_20220213T192202_A025585_T43PDR_N04.00
eo:cloud_cover: 0.001353
s2:datatake_id: GS2B_20220129T053049_025585_N04.00


In [78]:
dataset = CustomDataset(
    points_list,
    stac_item_list, 
    buffer=satellite_config['buffer_distance'],
    bands=satellite_config['bands'],
    resolution=satellite_config['resolution']
)


In [79]:
dataset[0]

TypeError: cannot unpack non-iterable NoneType object

In [57]:
data_loader = create_data_loader(points_gdf_with_stac, satellite_config, 1)

In [62]:
data_loader

In [68]:
examples = next(iter(data_loader))

In [69]:
examples

[None]

In [61]:
examples = next(iter(data_loader))
for label, img  in enumerate(examples):
   plt.imshow(img.permute(1,2,0))
   plt.show()
   print(f"Label: {label}")

0 Error: cannot unpack non-iterable NoneType object3 Error: cannot unpack non-iterable NoneType object4 Error: cannot unpack non-iterable NoneType object5 Error: cannot unpack non-iterable NoneType object2 Error: cannot unpack non-iterable NoneType object1 Error: cannot unpack non-iterable NoneType object


6 Error: cannot unpack non-iterable NoneType object
12 Error: cannot unpack non-iterable NoneType object



13 Error: cannot unpack non-iterable NoneType object11 Error: cannot unpack non-iterable NoneType object9 Error: cannot unpack non-iterable NoneType object

10 Error: cannot unpack non-iterable NoneType object

8 Error: cannot unpack non-iterable NoneType object


NameError: name 'plt' is not defined

14 Error: cannot unpack non-iterable NoneType object
16 Error: cannot unpack non-iterable NoneType object7 Error: cannot unpack non-iterable NoneType object

15 Error: cannot unpack non-iterable NoneType object
3 Error: cannot unpack non-iterable NoneType object1 Error: cannot unpack non-iterable NoneType object2 Error: cannot unpack non-iterable NoneType object0 Error: cannot unpack non-iterable NoneType object7 Error: cannot unpack non-iterable NoneType object4 Error: cannot unpack non-iterable NoneType object5 Error: cannot unpack non-iterable NoneType object




6 Error: cannot unpack non-iterable NoneType object

9 Error: cannot unpack non-iterable NoneType object11 Error: cannot unpack non-iterable NoneType object10 Error: cannot unpack non-iterable NoneType object12 Error: cannot unpack non-iterable NoneType object
15 Error: cannot unpack non-iterable NoneType object



8 Error: cannot unpack non-iterable NoneType object

16 Error: cannot unpack non-iterable NoneType object14 Er